In [ ]:
!pip install ensemble-boxes
from ensemble_boxes import *
import pandas as pd
from tqdm.auto import tqdm

# Load submissions

In [ ]:
sub1 = pd.read_csv("/kaggle/input/vinbigdata-final-submissions/final_submissions/vfnet.csv").sort_values(by="image_id").reset_index(drop=True)

sub2 = pd.read_csv("/kaggle/input/vinbigdata-final-submissions/final_submissions/vfnetx.csv").sort_values(by="image_id").reset_index(drop=True)

sub3 = pd.read_csv("/kaggle/input/vinbigdata-final-submissions/final_submissions/yolov5x.csv").sort_values(by="image_id").reset_index(drop=True)

sub4 = pd.read_csv("/kaggle/input/vinbigdata-final-submissions/final_submissions/yolov5x6.csv").sort_values(by="image_id").reset_index(drop=True)

In [ ]:
test_meta = pd.read_csv("/kaggle/input/vinbigdata-final-submissions/test_meta.csv").sort_values(by="image_id").reset_index(drop=True)
test_meta.head()

In [ ]:
submissions = [sub1, sub2, sub3, sub4]
weights = [1, 1, 1, 1]
iou_thr = 0.5
skip_box_thr = 0.08
sigma = 0.1

# Validate submissions

In [ ]:
# check for malformed submission outputs
for sub in submissions:    
    for item in sub.itertuples():
        assert len(item[2].split()) % 6 != 0

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
# check length of submissions
for sub in submissions:
    sub_preds = []
    for item in sub.itertuples():    
        sub_preds.append(list(chunks(item[2].split(), 6)))
    assert len(sub_preds) == 3000

# Ensemble predictions with WBF

In [ ]:
submission_list = []
num_boxes = []

for i, image_id in tqdm(enumerate(test_meta['image_id'])):
    # NOTE: dim0 is HEIGHT, dim1 is WIDTH
    height = int(test_meta[test_meta['image_id'] == image_id]['dim0'][i])
    width = int(test_meta[test_meta['image_id'] == image_id]['dim1'][i])
    boxes_list = []
    scores_list = []
    labels_list = []
    
    for sub in submissions:            
        temp_boxes = []
        temp_scores = []
        temp_labels = []
        for pred in list(chunks(sub['PredictionString'][i].split(), 6)):
            x_min = float(pred[2]) / width
            y_min = float(pred[3]) / height
            x_max = float(pred[4]) / width
            y_max = float(pred[5]) / height
            temp_boxes.append([x_min, y_min, x_max, y_max])
            temp_scores.append(float(pred[1]))
            temp_labels.append(int(pred[0]))
        
        boxes_list.append(temp_boxes)
        scores_list.append(temp_scores)
        labels_list.append(temp_labels)
        
    # perform WBF
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(width, height, width, height)
    boxes = boxes.round(1).tolist()
    labels = labels.astype(int).tolist()
    
    final_pred = []
    for j in range(len(boxes)):
        final_pred.append(str(labels[j]))
        final_pred.append(str(scores[j]))
        final_pred.append(str(boxes[j][0]))
        final_pred.append(str(boxes[j][1]))
        final_pred.append(str(boxes[j][2]))
        final_pred.append(str(boxes[j][3]))
    final_string = " ".join(final_pred)
    submission_list.append([image_id, final_string])
    
    num_boxes.append(len(boxes))

# Validate ensemble output

In [ ]:
# sanity check
submission_list[0]

In [ ]:
# view distribution of boxes per image
pd.Series(num_boxes).hist(bins=20)

# Create submission file

In [ ]:
df = pd.DataFrame(submission_list, columns = ['image_id','PredictionString'])
df.to_csv("submission.csv",index=False)
df.head()